# LAB | Imbalanced

**Load the data**

In this challenge, we will be working with Credit Card Fraud dataset.

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv

Metadata

- **distance_from_home:** the distance from home where the transaction happened.
- **distance_from_last_transaction:** the distance from last transaction happened.
- **ratio_to_median_purchase_price:** Ratio of purchased price transaction to median purchase price.
- **repeat_retailer:** Is the transaction happened from same retailer.
- **used_chip:** Is the transaction through chip (credit card).
- **used_pin_number:** Is the transaction happened by using PIN number.
- **online_order:** Is the transaction an online order.
- **fraud:** Is the transaction fraudulent. **0=legit** -  **1=fraud**


In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [2]:
fraud = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv")
fraud.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


**Steps:**

- **1.** What is the distribution of our target variable? Can we say we're dealing with an imbalanced dataset?
- **2.** Train a LogisticRegression.
- **3.** Evaluate your model. Take in consideration class importance, and evaluate it by selection the correct metric.
- **4.** Run **Oversample** in order to balance our target variable and repeat the steps above, now with balanced data. Does it improve the performance of our model? 
- **5.** Now, run **Undersample** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model?
- **6.** Finally, run **SMOTE** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model? 

In [3]:
#1. What is the distribution of our target variable? Can we say we're dealing with an imbalanced dataset?

# Ver la distribución de la variable objetivo "fraud"
print(fraud['fraud'].value_counts())

# Ver la proporción de cada clase
fraud['fraud'].value_counts(normalize=True)

fraud
0.0    912597
1.0     87403
Name: count, dtype: int64


fraud
0.0    0.912597
1.0    0.087403
Name: proportion, dtype: float64

In [4]:
# 2. Train a LogisticRegression.

# Separar las características (X) y la variable objetivo (y)
X = fraud.drop(columns=['fraud'])
y = fraud['fraud']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar un modelo de regresión logística
model = LogisticRegression()
model.fit(X_train_scaled, y_train)


LogisticRegression()

In [5]:
# 3. Evaluate your model. Take in consideration class importance, and evaluate it by selection the correct metric.

# Hacer predicciones
y_pred = model.predict(X_test_scaled)

# Matriz de confusión
print(confusion_matrix(y_test, y_pred))

# Reporte de clasificación
print(classification_report(y_test, y_pred))


[[181283   1274]
 [  6976  10467]]
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98    182557
         1.0       0.89      0.60      0.72     17443

    accuracy                           0.96    200000
   macro avg       0.93      0.80      0.85    200000
weighted avg       0.96      0.96      0.96    200000



In [6]:
# 4. Run Oversample in order to balance our target variable and repeat the steps above, now with balanced data. Does it improve the performance of our model?

# Aplicar Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Entrenar el modelo con los datos balanceados
model.fit(X_resampled, y_resampled)

# Evaluar el modelo
y_pred_resampled = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred_resampled))


              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96    182557
         1.0       0.58      0.95      0.72     17443

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000



In [7]:
# 5. Now, run Undersample in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model?

# Aplicar Undersampling
rus = RandomUnderSampler(random_state=42)
X_resampled_under, y_resampled_under = rus.fit_resample(X_train_scaled, y_train)

# Entrenar el modelo con los datos submuestreados
model.fit(X_resampled_under, y_resampled_under)

# Evaluar el modelo
y_pred_undersampled = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred_undersampled))


              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96    182557
         1.0       0.58      0.95      0.72     17443

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000



In [8]:
# 6. Finally, run SMOTE in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model?

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train_scaled, y_train)

# Entrenar el modelo con los datos balanceados por SMOTE
model.fit(X_resampled_smote, y_resampled_smote)

# Evaluar el modelo
y_pred_smote = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred_smote))


              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96    182557
         1.0       0.58      0.95      0.72     17443

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000



### **Respuestas a las preguntas del ejercicio:**

---

#### **1. What is the distribution of our target variable? Can we say we're dealing with an imbalanced dataset?**

El resultado muestra que la clase **fraud** (fraude) está altamente desbalanceada:
- **0.0 (no fraud):** 912,597 transacciones (91.26%)
- **1.0 (fraud):** 87,403 transacciones (8.74%)

Esto significa que sólo el 8.74% de las transacciones son fraudulentas, lo que indica que efectivamente estamos tratando con un conjunto de datos **desequilibrado**. Este desbalance es común en problemas de detección de fraudes, donde los casos fraudulentos son significativamente menores en comparación con los casos legítimos.

---

#### **2. Train a Logistic Regression**

El modelo de regresión logística se ha entrenado correctamente sin problemas. Este modelo es una elección común para problemas de clasificación binaria como el fraude. En este caso, se entrenó con los datos desbalanceados y, en los siguientes pasos, evaluaremos su desempeño.

---

#### **3. Evaluate your model**

Los resultados de la evaluación del modelo entrenado con datos desbalanceados son los siguientes:

- **Clase 0 (No fraude):** 
  - **Precisión:** 0.96
  - **Recall:** 0.99
  - **F1-score:** 0.98

- **Clase 1 (Fraude):**
  - **Precisión:** 0.89
  - **Recall:** 0.60
  - **F1-score:** 0.72

El modelo tiene un excelente desempeño al identificar transacciones **no fraudulentas** (alta precisión y recall para la clase 0). Sin embargo, su capacidad para detectar transacciones **fraudulentas** es notablemente más baja, con un **recall** del 60%. Esto sugiere que el modelo está teniendo dificultades para capturar todos los fraudes (muchos falsos negativos).

Dado que estamos trabajando con un problema de fraude, **recall** es especialmente importante porque queremos minimizar los casos en los que no detectamos fraudes (falsos negativos).

---

#### **4. Run Oversample**

Después de aplicar **Oversampling** (sobremuestreo) para balancear las clases, el rendimiento del modelo cambió de la siguiente manera:

- **Clase 0 (No fraude):**
  - **Precisión:** 1.00
  - **Recall:** 0.93
  - **F1-score:** 0.96

- **Clase 1 (Fraude):**
  - **Precisión:** 0.58
  - **Recall:** 0.95
  - **F1-score:** 0.72

El **recall** para la clase fraudulenta mejoró significativamente hasta el 95%, lo que significa que el modelo ahora está identificando casi todos los fraudes. Sin embargo, la **precisión** en la clase de fraude bajó al 58%, lo que implica más **falsos positivos** (transacciones legítimas clasificadas como fraudulentas). Aunque el modelo captura mejor los fraudes, se compromete la precisión.

---

#### **5. Run Undersample**

Al aplicar **Undersampling** (submuestreo), los resultados son casi idénticos a los del sobremuestreo. Esto puede deberse a que la cantidad de datos eliminados en la clase mayoritaria (no fraude) no afecta de manera significativa a la capacidad del modelo de detectar fraudes.

Los resultados:
- **Clase 0 (No fraude):** 1.00 precisión, 0.93 recall, 0.96 F1-score.
- **Clase 1 (Fraude):** 0.58 precisión, 0.95 recall, 0.72 F1-score.

Esto muestra que, aunque se equilibra la distribución, los problemas con la precisión para detectar fraudes persisten.

---

#### **6. Run SMOTE**

Finalmente, al aplicar **SMOTE** (que genera ejemplos sintéticos de la clase minoritaria), obtenemos resultados idénticos a los de las otras técnicas de balanceo:

- **Clase 0 (No fraude):** 1.00 precisión, 0.93 recall, 0.96 F1-score.
- **Clase 1 (Fraude):** 0.58 precisión, 0.95 recall, 0.72 F1-score.

SMOTE no parece ofrecer una mejora adicional en comparación con los otros métodos de balanceo.

---

### **Conclusión general**

- Con los datos desbalanceados, el modelo presenta un buen **recall** para la clase mayoritaria (no fraude), pero un rendimiento bajo para la clase minoritaria (fraude).
- Al aplicar técnicas de balanceo como **Oversampling**, **Undersampling**, y **SMOTE**, el **recall** para la clase fraudulenta mejora significativamente, pero a costa de una menor precisión, lo que significa más falsos positivos.
- **Oversampling**, **Undersampling**, y **SMOTE** producen resultados casi idénticos en este caso, y el uso de cualquiera de estas técnicas depende del contexto y los recursos computacionales disponibles.

Para problemas de fraude, donde es preferible capturar todos los fraudes incluso si se detectan algunos falsos positivos, estas técnicas de balanceo pueden ser útiles, pero es importante encontrar un equilibrio entre la precisión y el recall.